# 1. Package 설치

In [ ]:
%pip install langchain langchain-core langchain-community langchain-text-splitters langchain-openai langchain-pinecone

# 2. Knowledge Base 구성을 위한 데이터 생성

- Chroma를 활용한 2. LangChain과 Chroma를 활용한 RAG 구성과 동일함
- Vector Database만 Pinecone으로 변경

In [2]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./tax.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [3]:
document_list[52]

Document(metadata={'source': './tax.docx'}, page_content='⑤ 관할 세무서장이 제4항의 신청을 받았을 때에는 그 공제할 세액을 결정하여 신청인에게 알려야 한다.\n\n⑥ 제4항의 신청이 없는 경우에도 제1항을 적용한다.\n\n⑦ 집단적으로 재해가 발생한 경우에는 대통령령으로 정하는 바에 따라 관할 세무서장이 조사결정한 자산상실비율에 따라 제1항을 적용한다.\n\n⑧ 재해손실세액공제에 관하여 필요한 사항은 대통령령으로 정한다.\n\n[전문개정 2009. 12. 31.]\n\n\n\n제58조(재해손실세액공제) ① 사업자가 해당 과세기간에 천재지변이나 그 밖의 재해(이하 “재해”라 한다)로 대통령령으로 정하는 자산총액(이하 이 항에서 “자산총액”이라 한다)의 100분의 20 이상에 해당하는 자산을 상실하여 납세가 곤란하다고 인정되는 경우에는 다음 각 호의 소득세액(사업소득에 대한 소득세액을 말한다. 이하 이 조에서 같다)에 그 상실된 가액이 상실 전의 자산총액에서 차지하는 비율(이하 이 조에서 “자산상실비율”이라 한다)을 곱하여 계산한 금액(상실된 자산의 가액을 한도로 한다)을 그 세액에서 공제한다. 이 경우 자산의 가액에는 토지의 가액을 포함하지 아니한다. <개정 2020. 12. 29.>\n\n1. 재해 발생일 현재 부과되지 아니한 소득세와 부과된 소득세로서 미납된 소득세액\n\n2. 재해 발생일이 속하는 과세기간의 소득에 대한 소득세액\n\n② 제1항의 경우에 제56조, 제56조의2, 제57조 및 제57조의2에 따라 공제할 세액이 있을 때에는 이를 공제한 후의 세액을 소득세액으로 하여 제1항을 적용한다.<개정 2022. 12. 31.>\n\n③ 제1항에 따른 공제를 “재해손실세액공제”라 한다.\n\n④ 재해손실세액공제를 적용받으려는 자는 대통령령으로 정하는 바에 따라 관할 세무서장에게 신청할 수 있다.\n\n⑤ 관할 세무서장이 제4항의 신청을 받았을 때에는 그 공제할 세액을 결정하여 신청인에게 알려야 한다.\n\n⑥

In [4]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [6]:
import os

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = 'tax-index'
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)

In [7]:
query = '연봉 5천만원인 거주자의 종합소득세는?'

# 3. 답변 생성을 위한 Retrieval

- RetrievalQA에 전달하기 위해 retriever 생성
- search_kwargs의 k값을 변경해서 가져올 문서의 갯수를 지정할 수 있음
- .invoke()를 호출해서 어떤 문서를 가져오는지 확인 가능

In [8]:
retriever = database.as_retriever(search_kwargs={'k': 4})
retriever.invoke(query)

[Document(metadata={'source': './tax.docx'}, page_content='3. 「자본시장과 금융투자업에 관한 법률」 제251조제1항에 따른 집합투자업겸영보험회사의 특별계정\n\n③ 비거주자의 소득은 제119조에 따라 구분한다.\n\n[전문개정 2009. 12. 31.]\n\n\n\n제4조(소득의 구분) ① 거주자의 소득은 다음 각 호와 같이 구분한다. <개정 2013. 1. 1., 2020. 12. 29.>\n\n1. 종합소득\n\n\u3000이 법에 따라 과세되는 모든 소득에서 제2호, 제2호의2 및 제3호에 따른 소득을 제외한 소득으로서 다음 각 목의 소득을 합산한 것\n\n가. 이자소득\n\n나. 배당소득\n\n다. 사업소득\n\n라. 근로소득\n\n마. 연금소득\n\n바. 기타소득\n\n2. 퇴직소득\n\n2의2. 금융투자소득\n\n3. 양도소득\n\n② 제1항에 따른 소득을 구분할 때 다음 각 호의 신탁을 제외한 신탁의 이익은 「신탁법」 제2조에 따라 수탁자에게 이전되거나 그 밖에 처분된 재산권에서 발생하는 소득의 내용별로 구분한다.<개정 2011. 7. 25., 2020. 12. 29., 2022. 12. 31.>\n\n1. 「법인세법」 제5조제2항에 따라 신탁재산에 귀속되는 소득에 대하여 그 신탁의 수탁자가 법인세를 납부하는 신탁\n\n2. 「자본시장과 금융투자업에 관한 법률」 제9조제18항제1호에 따른 투자신탁. 다만, 2024년 12월 31일까지는 이 법 제17조제1항제5호에 따른 집합투자기구로 한정한다.\n\n3. 「자본시장과 금융투자업에 관한 법률」 제251조제1항에 따른 집합투자업겸영보험회사의 특별계정\n\n③ 비거주자의 소득은 제119조에 따라 구분한다.\n\n[전문개정 2009. 12. 31.]\n\n[시행일: 2025. 1. 1.] 제4조제1항제1호, 제4조제1항제2호의2\n\n\n\n제5조(과세기간) ① 소득세의 과세기간은 1월 1일부터 12월 31일까지 1년으로 한다.\n\n② 거주자가 사망한 경우

# 4. Augmentation을 위한 Prompt 활용

- Retrieval 된 데이터는 LangChain에서 제공하는 프롬프트("rlm/rag-prompt") 사용

In [9]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [10]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

# 5. 답변 생성

- RetrievalQA를 통해 LLM에 전달
    - RetrievalQA는 create_retrieval_chain으로 대체됨
    - 실제 ChatBot 구현 시 create_retrieval_chain으로 변경하는 과정을 볼 수 있음

In [11]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

In [12]:
ai_message = qa_chain.invoke({"query": query})

In [13]:
ai_message

{'query': '연봉 5천만원인 거주자의 종합소득세는?',
 'result': '연봉 5천만원인 거주자의 종합소득세는 약 410만원입니다.'}